In [14]:
from transformers import AutoModel, AutoTokenizer, pipeline
from forgebox.imports import *
from ipywidgets import Textarea, Layout
from spacy import load
from tqdm.notebook import tqdm
from forgebox.ftorch.cuda import CudaHandler

In [15]:
dev = CudaHandler().idle().device

>>> 2 cuda devices found >>>
Device 0: 
	name:Tesla V100-PCIE-32GB
	used:31019MB	free:1491MB
Device 1: 
	name:Tesla V100-PCIE-32GB
	used:4MB	free:32506MB
cuda stats refreshed
Found the most idle GPU: cuda:1, 32506 MB Mem remained


## Paraphrasing

In [2]:
nlp = load("en_ner_bionlp13cg_md")

In [19]:
tag = "tuner007/pegasus_paraphrase"
paraphraser = pipeline(model=tag, tokenizer=tag, task="text2text-generation", device=dev.index)

In [20]:
def rephrase(text):
    doc = nlp(text)
    return " ".join(pd.DataFrame(
        paraphraser(list((map(lambda x:x.text, doc.sents))), truncation=True,)
    )["generated_text"])

In [21]:
train_df = pd.read_csv("train.csv")

In [22]:
def rewrite(df):
    results = []
    for excerpt in tqdm(list(df.excerpt)):
        results.append(rephrase(excerpt))
    df2 = df.copy()
    df2["excerpt"] = results
    return df2

In [23]:
rewrite(train_df).to_csv("aug_train.csv", index=False)

  0%|          | 0/2834 [00:00<?, ?it/s]

### Testing

In [24]:
aug1 = pd.read_csv("aug_train.csv")

In [27]:
(aug1.excerpt==train_df.excerpt).sum()

0

## GPT2